# 读取数据

In [1]:
import numpy as np
from pyscipopt import Model, quicksum, multidict

# 读取数据
class UncapacitatedWarehouseLocation():
    def __init__(self):# 确定读取文件的路径
        self.dir = "cap71.txt"

    def read(self):  #读取数据
        dir = self.dir
        f = open(dir, "r")
        lines = f.readlines() #获取每行数据
        f.close()  
        i = 0
        warehouseNum = 0
        customerNum = 0
        Q = [] #容量
        F = [] #固定建设费用
        D = [] #各客户点需求量
        C = [] #各客户点到仓库费用
        for s in lines:
            if i == 0: #第一行读取客户数量和仓库数量
                line = s.split()
                warehouseNum = int(line[0])
                customerNum = int(line[1])
                i += 1
                continue
            if i <= warehouseNum:#读取仓库的容量和费用
                a, b = s.split()
                Q.append(int(a))
                F.append(float(b))
                i += 1
                continue
            si = s.split()
            if len(si) == 1: #读取客户需求量
                Di = int(si[0])
                D.append(Di)
                continue
            line = [float(a) for a in si] #读取客户到各仓库的费用
            C.extend(line)
        C = np.array(C).reshape(customerNum,warehouseNum)
        C = C.T# 进行转置
        return warehouseNum, customerNum, F, D, C #返回仓库数量，客户数量，固定建设费用，需求量，花费

# SCIP求解代码

In [20]:
def solve_by_cplex(F, C):
    WN, NN = C.shape[0], C.shape[1] #行维度为仓库数量，列维度为客户数量
    W = range(WN)
    N = range(NN)

    model = Model("Uncapacitated warehouse location")

#对每个客户的分配以及仓库是否建立设定0-1变量，这里给出的是scip的约束构造方法
    x, y = {}, {}
    for i in W:
        y[i] = model.addVar(vtype="B", name="y(%s)"%i)
        for j in N:
            x[i, j] = model.addVar(vtype="B", name="x(%s, %s)"%(i,j))

    model.setObjective(quicksum(C[i, j]*x[i, j] for i in W for j in N)+quicksum(F[i]*y[i] for i in W), "minimize") #目标函数

    for j in N:  #逻辑约束
        model.addCons(quicksum(x[i,j] for i in W) == 1)
        for i in W:
            model.addCons(x[i,j] <= y[i])

    model.redirectOutput()
    model.optimize() #优化
    A = [0] * NN  #存放求解结果
    for i in W:
        if model.getVal(y[i]) > 0.1:
            for j in N:
                if model.getVal(x[i,j]) >= 0.01:
                    A[j] = i
    return model.getObjVal(), A

# 启发式算法

In [37]:
def cost(A, F, L):  #计算当前解的值
    N = range(len(A))# 获取客户数量
    H = set(A) #获取被选中的仓库集合
    c = sum(L[A[i], i] for i in N)  #计算各客户到仓库的成本
    c += sum(F[i] for i in H) #再加上建设费用
    return c

def get_A(H, L):  # 获取初始可行解
    N = L.shape[1] #获取客户数量
    A = []  
    for j in range(N): #遍历客户
        D = [L[i, j] for i in H] #获取每个客户到所有仓库的费用列表
        min_id = D.index(min(D)) #获取最少费用的那个仓库index
        A.append(H[min_id]) #初始解对每个客户分配一个初始仓库点
    return A

def heuristicAlgorithm(F, C): #启发式算法
    WN, NN = C.shape[0], C.shape[1]  #获取仓库和客户数量
    cmin = np.inf #设置最小值为无穷大
    n = 0 
    Amin = None 
    while n < 500:# 迭代100次
        wi = np.random.randint(WN / 2, WN) #以下两步随机生成候选仓库列表
        H = np.random.choice(WN, wi, replace=False)  #从[0,WN)范围内选取wi个随机数，replace=False时表示数字不能重复
        A = get_A(H, C)  #获得可行解A
        c = cost(A, F, C) #计算费用目标函数
        if c < cmin: #当费用低于当前最小值时，进行重新赋值
            cmin = c
            Amin = A #更新最优解
            print("{}:{}".format(n, c))
        n += 1 
    return cmin, Amin

# 输出结果

In [22]:
def Report(c, A):
    A = np.array(A)
    P = set(A)
    print("*" * 50)
    print("The optimal objective is %g" % c)
    for i in P: #输出每个仓库对接的客户编号
        print("Warehouse location {}:".format(i), end="")
        pi = np.where(A == i) 
        print(list(pi[0]))
    print("*" * 50)

In [23]:
def test_1():
    CWL = UncapacitatedWarehouseLocation()
    WN, NN, F, D, C = CWL.read()

    # SCIP求解
    c, A = solve_by_cplex(F, C)
    Report(c, A)

In [24]:
def test_2():
    CWL = UncapacitatedWarehouseLocation()
    WN, NN, F, D, C = CWL.read()

    # 启发式算法求解
    ch, Ah = heuristicAlgorithm(F, C)
    Report(ch, Ah)

In [26]:
test_1()

presolving:
(round 1, fast)       1 del vars, 50 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 800 clqs
   (0.0s) running MILP presolver
   (0.0s) MILP presolver (2 rounds): 0 aggregations, 624 fixings, 0 bound changes
(round 2, medium)     625 del vars, 850 del conss, 176 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 176 clqs
(round 3, fast)       648 del vars, 873 del conss, 176 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 153 clqs
(round 4, fast)       650 del vars, 888 del conss, 176 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 138 clqs
   (0.0s) running MILP presolver
   (0.0s) MILP presolver (2 rounds): 0 aggregations, 22 fixings, 0 bound changes
(round 5, medium)     672 del vars, 888 del conss, 176 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 132 clqs
(round 6, fast)       676 del vars, 898 del conss, 176 add conss, 0 c

In [39]:
test_2()#启发式算法求解

0:959016.5874999999
7:951260.6124999999
11:943764.3999999999
40:940386.1
226:939949.4124999999
330:938158.1124999999
340:933876.2999999999
**************************************************
The optimal objective is 933876
Warehouse location 0:[2, 5, 13, 23, 29, 30, 32]
Warehouse location 1:[6]
Warehouse location 2:[7, 33]
Warehouse location 3:[10, 16, 18, 20, 35, 41]
Warehouse location 5:[3, 12, 36, 37, 39, 48]
Warehouse location 6:[1, 14, 19, 21, 34, 43, 47]
Warehouse location 7:[0, 4, 8, 9, 15, 38, 42, 45, 46]
Warehouse location 8:[17]
Warehouse location 10:[11, 22, 24, 25, 27, 28, 31, 40, 49]
Warehouse location 12:[26, 44]
**************************************************


In [19]:
wi=np.random.randint(4,8)
H=np.random.choice(8,wi,replace=False)
H

array([0, 7, 2, 1])